In [1]:
import sys
import os

# Set project root as module path
sys.path.append(os.path.abspath(".."))

In [2]:
# load api keys from .env file 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Setup: Import all modules
from src.chunking import load_and_chunk

from src.embedding import get_embedding_model
from src.vector_store import build_vectorstore, save_vectorstore, load_vectorstore
from src.query_api import build_qa_chain


/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Step 1: Load and chunk PDF
pdf_path = "../data/sample.pdf"  # change path if needed
chunks = load_and_chunk(pdf_path)
print(f"Total chunks: {len(chunks)}")
print(chunks[0])
print("\n\n---\n\n")
print(chunks[1])


Total chunks: 11
page_content='Christopher Kauffman
Associate Teaching Professor Ofﬁce: 612-626-9351
Department of Computer Science Mobile: 612-396-4971
University of Minnesota0571A Email: kauffman@umn.edu
200 Union St SE Room4-192 URL: http://cs.umn.edu/~kauffman
Minneapolis, MN55455 USA
Education
Ph.D. Computer Science, University of Minnesota, Twin Cities.2013.
Dissertation: Computational Methods for Protein Structure Prediction and Energy Minimization.
M.S. Computer Science, University of Minnesota, Twin Cities,2010.' metadata={'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'LaTeX with hyperref', 'creationdate': "D:20250728060413Z00'00'", 'title': 'Christopher Kauffman: Curriculum Vitae', 'author': 'Christopher Kauffman', 'moddate': "D:20250728060413Z00'00'", 'source': '../data/sample.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}


---


page_content='B.S. Computer Science, University of Minnesota, Twin Cities,2004.
Minor in Mathematics. Gradua

In [5]:
# Step 2: Build embedding model
embedding_model = get_embedding_model()

/Users/yinglu/Desktop/projects/pdf-rag-qa/src/embedding.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(


In [6]:
# Step 3: Build vector store
texts = [doc.page_content for doc in chunks]
metadatas = [doc.metadata for doc in chunks]
vectorstore = build_vectorstore(texts, metadatas, embedding_model=embedding_model)


/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
# Step 4: Save vector store locally
store_path = "../faiss_store"
save_vectorstore(vectorstore, path=store_path)

In [8]:
# Step 5: Load vector store and build QA chain
vs = load_vectorstore(path=store_path, embedding_model=embedding_model)


In [16]:
qa_chain = build_qa_chain(vs)

# Step 6: Ask a question and get answer
query = "what is the email address of this person?"
#print(qa_chain.prompt.format_prompt(question=query, context="...").to_string())

result = qa_chain(query)

print("Answer:\n", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(" -", doc.metadata)

Device set to use mps:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/Users/yinglu/Desktop/projects/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Token indices sequence length is longer than the specified maximum sequence length for this model (679 > 512). Running this sequence through the model will result in indexing errors
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer:
 kauffman@umn.edu

Sources:
 - {'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'LaTeX with hyperref', 'creationdate': "D:20250728060413Z00'00'", 'title': 'Christopher Kauffman: Curriculum Vitae', 'author': 'Christopher Kauffman', 'moddate': "D:20250728060413Z00'00'", 'source': '../data/sample.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}
 - {'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'LaTeX with hyperref', 'creationdate': "D:20250728060413Z00'00'", 'title': 'Christopher Kauffman: Curriculum Vitae', 'author': 'Christopher Kauffman', 'moddate': "D:20250728060413Z00'00'", 'source': '../data/sample.pdf', 'total_pages': 2, 'page': 1, 'page_label': '2'}
 - {'producer': 'macOS Version 15.5 (Build 24F74) Quartz PDFContext', 'creator': 'LaTeX with hyperref', 'creationdate': "D:20250728060413Z00'00'", 'title': 'Christopher Kauffman: Curriculum Vitae', 'author': 'Christopher Kauffman', 'moddate': "D:20250728060413Z00'